<a href="https://colab.research.google.com/github/puraminy/mini_proj3/blob/master/dcgan_linux.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
# -*- coding: utf-8 -*-
"""Main-DCGAN.ipynb

Automatically generated by Colaboratory.

Original file is located at
    https://colab.research.google.com/github/puraminy/mini_proj3/blob/master/Main_DCGAN.ipynb
"""

#from keras.datasets import mnist
from keras.models import Model, Sequential
from keras.layers import *
from keras.optimizers import Adam
from tqdm import tqdm
from keras.layers.advanced_activations import LeakyReLU
import numpy as np
import matplotlib.pyplot as plt
# %matplotlib inline
import tensorflow as tf
# GPU Processing
#device_name = tf.test.gpu_device_name()
#if device_name != '/device:GPU:0':
#  raise SystemError('GPU device not found')
#print('Found GPU at:{}'.format(device_name))
#import pickle
import _pickle as cPickle
import sys
import gzip

dataset = 'mnist.pkl.gz'
f = gzip.open(dataset, 'rb')
if sys.version_info < (3,):
    training_data, validation_data, test_data = cPickle.load(f)
else:
    training_data, validation_data, test_data = cPickle.load(f, encoding='bytes')
f.close()

#sys.stdout = open('dcgan.out', 'w')

from contextlib import redirect_stdout

with open('help.txt', 'w') as f:
    with redirect_stdout(f):
        print('it now prints to `help.text`')


X_train, Y_train = training_data[0], training_data[1]

X_test, Y_test = test_data[0], test_data[1]

X_train = X_train[:30000,:]
Y_train = Y_train[:30000]
X_test  = X_test[:5000,:]
Y_test  = Y_test[:5000]

X_train.shape, Y_train.shape, X_test.shape, Y_test.shape

X_train = X_train.reshape(X_train.shape[0], 28, 28, 1)
X_test = X_test.reshape(X_test.shape[0], 28, 28, 1)

X_train = X_train.astype('float32')

# Scaling the range of the image to [-1, 1]
# Because we are using tanh as the activation function in the last layer of the generator
# and tanh restricts the weights in the range [-1, 1]
X_train = (X_train - 127.5) / 127.5

X_train.shape

def build_generator(input_dim):

    model = Sequential()

    model.add(Dense(128 * 7 * 7, activation="relu", input_dim=input_dim))
    model.add(Reshape((7, 7, 128)))
    model.add(UpSampling2D())
    model.add(Conv2D(128, kernel_size=3, padding="same"))
    model.add(BatchNormalization(momentum=0.8))
    model.add(Activation("relu"))
    model.add(UpSampling2D())
    model.add(Conv2D(64, kernel_size=3, padding="same"))
    model.add(BatchNormalization(momentum=0.8))
    model.add(Activation("relu"))
    model.add(Conv2D(1, kernel_size=3, padding="same"))
    model.add(Activation("tanh"))

    return model

def build_discriminator(input_shape):

    model = Sequential()

    model.add(Conv2D(32, kernel_size=3, strides=2, input_shape=input_shape, padding="same"))
    model.add(LeakyReLU(alpha=0.2))
    model.add(Dropout(0.25))
    model.add(Conv2D(64, kernel_size=3, strides=2, padding="same"))
    model.add(ZeroPadding2D(padding=((0,1),(0,1))))
    model.add(BatchNormalization(momentum=0.8))
    model.add(LeakyReLU(alpha=0.2))
    model.add(Dropout(0.25))
    model.add(Conv2D(128, kernel_size=3, strides=2, padding="same"))
    model.add(BatchNormalization(momentum=0.8))
    model.add(LeakyReLU(alpha=0.2))
    model.add(Dropout(0.25))
    model.add(Conv2D(256, kernel_size=3, strides=1, padding="same"))
    model.add(BatchNormalization(momentum=0.8))
    model.add(LeakyReLU(alpha=0.2))
    model.add(Dropout(0.25))
    model.add(Flatten())
    model.add(Dense(1, activation='sigmoid'))

    return model

generator = Sequential([
        Dense(128*7*7, input_dim=100, activation=LeakyReLU(0.2)),
        BatchNormalization(),
        Reshape((7,7,128)),
        UpSampling2D(),
        Convolution2D(64, 5, 5, border_mode='same', activation=LeakyReLU(0.2)),
        BatchNormalization(),
        UpSampling2D(),
        Convolution2D(1, 5, 5, border_mode='same', activation='tanh')
    ])

# generator = build_generator(input_dim=100)

generator.summary()

discriminator = Sequential([
        Convolution2D(64, 5, 5, subsample=(2,2), input_shape=(28,28,1), border_mode='same', activation=LeakyReLU(0.2)),
        Dropout(0.3),
        Convolution2D(128, 5, 5, subsample=(2,2), border_mode='same', activation=LeakyReLU(0.2)),
        Dropout(0.3),
        Flatten(),
        Dense(1, activation='sigmoid')
    ])

# discriminator = build_discriminator(input_shape=(28,28,1))

discriminator.summary()

generator.compile(loss='binary_crossentropy', optimizer=Adam(0.0002, 0.5))

discriminator.compile(loss='binary_crossentropy', optimizer=Adam(0.0002, 0.5))

discriminator.trainable = False
ganInput = Input(shape=(100,))
# getting the output of the generator
# and then feeding it to the discriminator
# new model = D(G(input))
x = generator(ganInput)
ganOutput = discriminator(x)
gan = Model(input=ganInput, output=ganOutput)
gan.compile(loss='binary_crossentropy', optimizer=Adam())

gan.summary()

def save_imgs(epoch, save_folder='.'):
    r, c = 5, 5
    noise = np.random.normal(0, 1, (r * c, 100))
    gen_imgs = generator.predict(noise)

    # Rescale images 0 - 1
    gen_imgs = 0.5 * gen_imgs + 0.5

    fig, axs = plt.subplots(r, c)
    cnt = 0
    for i in range(r):
        for j in range(c):
            axs[i,j].imshow(gen_imgs[cnt, :,:,0], cmap='gray')
            axs[i,j].axis('off')
            cnt += 1
    fig.savefig(save_folder + "/mnist_%d.png" % epoch)
    plt.show()
    plt.close()
    
def show_imgs(epoch, predictions, save_folder='.'):
    r, c = 5, 5
    fig, axs = plt.subplots(r, c)
    cnt = 0
    for i in range(r):
        for j in range(c):
            axs[i,j].imshow(predictions[cnt, :,:,0], cmap='gray')
            axs[i,j].axis('off')
            cnt += 1
    fig.savefig(save_folder + "/mnist_%d.png" % epoch)
    plt.show()
    plt.close()    
    
def plot_output(num=10):
    try_input = np.random.rand(100, 100)
    preds = generator.predict(try_input)
    
    show_imgs(10, preds)
#     plt.figure(figsize=(10,10))
#     for i in range(preds.shape[0]):
#         plt.subplot(num, num, i+1)
#         plt.imshow(preds[i, :, :, 0], cmap='gray')
#         plt.axis('off')
    
#     # tight_layout minimizes the overlap between 2 sub-plots
#     plt.tight_layout()  
#     plt.show()

import time
import progressbar as pg
#from laplotter import LossAccPlotter
import numpy as np


def train(epochs=10, batch_size=128):
    batch_count = X_train.shape[0] // batch_size

    dloss = []
    gloss = []
    
    t=0
#    batch_plotter = LossAccPlotter(show_regressions=True,
#                         show_averages=True,
#                         show_loss_plot=True)
#    epoch_plotter = LossAccPlotter(show_regressions=True,
#                         show_averages=True,
#                         show_loss_plot=True)

    for i in range(epochs):
        for j in pg.progressbar(range(batch_count)):
            time.sleep(0.02)
            # Input for the generator
            noise_input = np.random.rand(batch_size, 100)

#             noise_input = np.random.normal(0, 1, (batch_size, 100))
            # getting random images from X_train of size=batch_size 
            # these are the real images that will be fed to the discriminator
            image_batch = X_train[np.random.randint(0, X_train.shape[0], size=batch_size)]

            # these are the predicted images from the generator
            predictions = generator.predict(noise_input, batch_size=batch_size)

            # the discriminator takes in the real images and the generated images
            X = np.concatenate([predictions, image_batch])

            # labels for the discriminator
            y_discriminator = [0] * batch_size + [1] * batch_size

            # Let's train the discriminator
            discriminator.trainable = True
            d_loss = discriminator.train_on_batch(X, y_discriminator)

            # Let's train the generator
            noise_input = np.random.rand(batch_size, 100)
            y_generator = [1] * batch_size
            discriminator.trainable = False

            g_loss = gan.train_on_batch(noise_input, y_generator)

            gloss.append(g_loss)
            dloss.append(d_loss)
 #           batch_plotter.add_values(t, loss_val=g_loss, loss_train=d_loss)
 #           t+=1
        #             print("%d [D loss: %f] [G loss: %f]" % (j, d_loss, g_loss))
 #           if j % 100 == 0:
 #       epoch_plotter.add_values(i, loss_val=g_loss, loss_train=d_loss)
        print("Epoch:", i)
        save_imgs(i, './dcgan')
        
#    epoch_plotter.block()
#    batch_plotter.block()
#    plt.show()
    return gloss, dloss

gloss, dloss = train(epochs=2, batch_size=512)

# plt.figure(num=2, figsize=(20, 40), dpi=80, facecolor='w', edgecolor='k')
# plt.subplots_adjust(hspace=0.3,bottom=0.3)

# ax = plt.subplot2grid((5,2),(0,0), colspan=1)
# ax.set_title("loss")

# # ax1.plot(zip(range(len(gloss)),gloss), label='train')
# ax.plot(gloss)
# ax.set_xlabel("Num of Epochs")
# ax.set_ylabel("Loss")

# plt.show()


with open('dcgan/gloss.txt', 'w') as f:
    for item in gloss:
        f.write("%s\n" % item)

with open('dcgan/dloss.txt', 'w') as f:
    for item in dloss:
        f.write("%s\n" % item)
# generate some example values for the loss training line

generator.save_weights('dcgan\gen_20_scaled_images.h5')
discriminator.save_weights('dcgan\dis_20_scaled_images.h5')


plot_output()



